# NFL Player Clustering By Position

## Data Pre-Processing

In [24]:
import pandas as pd
import numpy as np

In [9]:
passing_basic_stats = pd.read_csv("https://raw.githubusercontent.com/jake-lukasik/NFL-Player-Clustering/refs/heads/main/pro-football-reference-data/Passing-BasicStats.csv")
passing_basic_stats.head()

,Rk,Player,Age,Team,Pos,G,GS,QBrec,Cmp,Att,...,QBR,Sk,Yds.1,Sk%,NY/A,ANY/A,4QC,GWD,Awards,Player-additional
0,1.0,Joe Burrow,28.0,CIN,QB,17.0,17.0,9-8-0,460.0,652.0,...,74.7,48.0,278.0,6.86,6.63,7.28,1.0,2.0,PBAP MVP-4AP OPoY-5AP CPoY-1,BurrJo01
1,2.0,Jared Goff,30.0,DET,QB,17.0,17.0,15-2-0,390.0,539.0,...,68.4,31.0,234.0,5.44,7.71,8.06,4.0,4.0,PBAP MVP-5AP OPoY-9,GoffJa00
2,3.0,Baker Mayfield,29.0,TAM,QB,17.0,17.0,10-7-0,407.0,570.0,...,61.6,40.0,248.0,6.56,6.97,7.13,2.0,2.0,PBAP MVP-11AP OPoY-11,MayfBa00
3,4.0,Geno Smith,34.0,SEA,QB,17.0,17.0,10-7-0,407.0,578.0,...,53.8,50.0,338.0,7.96,6.34,5.93,4.0,4.0,NaN,SmitGe00
4,5.0,Sam Darnold,27.0,MIN,QB,17.0,17.0,14-3-0,361.0,545.0,...,60.4,48.0,335.0,8.09,6.72,6.99,3.0,5.0,PBAP MVP-10AP CPoY-3,DarnSa00


In [10]:
rushing_basic_stats = pd.read_csv("https://raw.githubusercontent.com/jake-lukasik/NFL-Player-Clustering/refs/heads/main/pro-football-reference-data/Rushing-BasicStats.csv")
rushing_basic_stats.head()

,Rk,Player,Age,Team,Pos,G,GS,Att,Yds,TD,1D,Succ%,Lng,Y/A,Y/G,A/G,Fmb,Awards,-9999
0,1.0,Saquon Barkley,27.0,PHI,RB,16.0,16.0,345.0,2005.0,13.0,82.0,52.5,72.0,5.8,125.3,21.6,2.0,PBAP-1AP MVP-3AP OPoY-1AP CPoY-14,BarkSa00
1,2.0,Derrick Henry,30.0,BAL,RB,17.0,17.0,325.0,1921.0,16.0,94.0,58.8,87.0,5.9,113.0,19.1,3.0,PBAP-2AP OPoY-4,HenrDe00
2,3.0,Kyren Williams,24.0,LAR,RB,16.0,16.0,316.0,1299.0,14.0,85.0,50.9,30.0,4.1,81.2,19.8,5.0,AP OPoY-11,WillKy02
3,4.0,Bijan Robinson,22.0,ATL,RB,17.0,17.0,304.0,1456.0,14.0,82.0,60.2,37.0,4.8,85.6,17.9,1.0,PB,RobiBi01
4,5.0,Jonathan Taylor,25.0,IND,RB,14.0,13.0,303.0,1431.0,11.0,71.0,45.9,70.0,4.7,102.2,21.6,4.0,PBAP CPoY-19,TaylJo02


In [11]:
receiving_basic_stats = pd.read_csv("https://raw.githubusercontent.com/jake-lukasik/NFL-Player-Clustering/refs/heads/main/pro-football-reference-data/Receiving-BasicStats.csv")
receiving_basic_stats.head()

,Rk,Player,Age,Team,Pos,G,GS,Tgt,Rec,Yds,...,1D,Succ%,Lng,R/G,Y/G,Ctch%,Y/Tgt,Fmb,Awards,-9999
0,1.0,Ja'Marr Chase,24.0,CIN,WR,17.0,16.0,175.0,127.0,1708.0,...,75.0,62.3,70.0,7.5,100.5,72.6,9.8,0.0,PBAP-1AP MVP-8AP OPoY-3,ChasJa00
1,2.0,Amon-Ra St. Brown,25.0,DET,WR,17.0,17.0,141.0,115.0,1263.0,...,73.0,70.2,66.0,6.8,74.3,81.6,9.0,1.0,PBAP-1AP OPoY-11,StxxAm00
2,3.0,Brock Bowers,22.0,LVR,TE,17.0,16.0,153.0,112.0,1194.0,...,61.0,60.8,57.0,6.6,70.2,73.2,7.8,0.0,PBAP-1AP ORoY-2,BoweBr01
3,4.0,Trey McBride,25.0,ARI,TE,16.0,16.0,147.0,111.0,1146.0,...,63.0,63.3,37.0,6.9,71.6,75.5,7.8,0.0,PB,McBrTr01
4,5.0,Malik Nabers,21.0,NYG,WR,15.0,13.0,170.0,109.0,1204.0,...,55.0,51.2,59.0,7.3,80.3,64.1,7.1,1.0,PBAP ORoY-5,NabeMa00


# Processing QB Stats:
## Filter out unnecessary cols & get wins from record

In [12]:
passing_basic_stats.columns

Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'QBrec', 'Cmp', 'Att',
       'Cmp%', 'Yds', 'TD', 'TD%', 'Int', 'Int%', '1D', 'Succ%', 'Lng', 'Y/A',
       'AY/A', 'Y/C', 'Y/G', 'Rate', 'QBR', 'Sk', 'Yds.1', 'Sk%', 'NY/A',
       'ANY/A', '4QC', 'GWD', 'Awards', 'Player-additional'],
      dtype='object')

In [13]:
def get_wins(record):
    if isinstance(record, str) and '-' in record:
        split_rec = record.split('-')
        return int(split_rec[0])
    return 0

In [14]:
passing_basic_stats = passing_basic_stats.drop(columns=['Rk', 'Cmp', 'Att', 'Sk', 'Yds.1', 'Sk%', 'Awards', 'Player-additional'])
# (Drop sack related stats b/c I'm a Bears fan and those are NOT QB Stats :P )
passing_basic_stats = passing_basic_stats[passing_basic_stats['Pos'] == 'QB']
passing_basic_stats = passing_basic_stats[passing_basic_stats['GS'] >= 9]
passing_basic_stats['QB-Wins'] = passing_basic_stats['QBrec'].apply(get_wins)
passing_basic_stats = passing_basic_stats.drop(columns=['QBrec', 'Pos'])
passing_basic_stats.head()

,Player,Age,Team,G,GS,Cmp%,Yds,TD,TD%,Int,...,AY/A,Y/C,Y/G,Rate,QBR,NY/A,ANY/A,4QC,GWD,QB-Wins
0,Joe Burrow,28.0,CIN,17.0,17.0,70.6,4918.0,43.0,6.6,9.0,...,8.24,10.7,289.3,108.5,74.7,6.63,7.28,1.0,2.0,9
1,Jared Goff,30.0,DET,17.0,17.0,72.4,4629.0,37.0,6.9,12.0,...,8.96,11.9,272.3,111.8,68.4,7.71,8.06,4.0,4.0,15
2,Baker Mayfield,29.0,TAM,17.0,17.0,71.4,4500.0,41.0,7.2,16.0,...,8.07,11.1,264.7,106.8,61.6,6.97,7.13,2.0,2.0,10
3,Geno Smith,34.0,SEA,17.0,17.0,70.4,4320.0,21.0,3.6,15.0,...,7.03,10.6,254.1,93.2,53.8,6.34,5.93,4.0,4.0,10
4,Sam Darnold,27.0,MIN,17.0,17.0,66.2,4319.0,35.0,6.4,12.0,...,8.22,12.0,254.1,102.5,60.4,6.72,6.99,3.0,5.0,14


## Isolate the columns into numeric and non-numeric features for PCA:

In [ ]:
NFL_QB_Identifiers = passing_basic_stats[['Player', 'Team']]
NFL_QB_Features = passing_basic_stats.drop(columns=['Player', 'Team'])

#NFL_QB_Identifiers.to_csv("C:/Users/jakel/OneDrive/Desktop/GitHub/NFL-Player-Clustering/filtered-data/NFL-QB-Identifiers.csv")
#NFL_QB_Features.to_csv("C:/Users/jakel/OneDrive/Desktop/GitHub/NFL-Player-Clustering/filtered-data/NFL-QB-Features.csv")

# Processing Rushing Stats:
## Filter out unnecessary cols

In [20]:
rushing_basic_stats.columns

Index(['Rk', 'Player', 'Age', 'Team', 'Pos', 'G', 'GS', 'Att', 'Yds', 'TD',
       '1D', 'Succ%', 'Lng', 'Y/A', 'Y/G', 'A/G', 'Fmb', 'Awards', '-9999'],
      dtype='object')

In [21]:
rushing_basic_stats = rushing_basic_stats.drop(columns=["Rk", "Awards", "-9999"])
rushing_basic_stats = rushing_basic_stats[rushing_basic_stats["Pos"] == "RB"]
rushing_basic_stats = rushing_basic_stats[rushing_basic_stats["GS"] >= 9] # Use same criteria we used for QBs
rushing_basic_stats = rushing_basic_stats.drop(columns=["Pos"])
rushing_basic_stats.head()

,Player,Age,Team,G,GS,Att,Yds,TD,1D,Succ%,Lng,Y/A,Y/G,A/G,Fmb
0,Saquon Barkley,27.0,PHI,16.0,16.0,345.0,2005.0,13.0,82.0,52.5,72.0,5.8,125.3,21.6,2.0
1,Derrick Henry,30.0,BAL,17.0,17.0,325.0,1921.0,16.0,94.0,58.8,87.0,5.9,113.0,19.1,3.0
2,Kyren Williams,24.0,LAR,16.0,16.0,316.0,1299.0,14.0,85.0,50.9,30.0,4.1,81.2,19.8,5.0
3,Bijan Robinson,22.0,ATL,17.0,17.0,304.0,1456.0,14.0,82.0,60.2,37.0,4.8,85.6,17.9,1.0
4,Jonathan Taylor,25.0,IND,14.0,13.0,303.0,1431.0,11.0,71.0,45.9,70.0,4.7,102.2,21.6,4.0


In [28]:
rushing_eng_stats = rushing_basic_stats # clone for a checkpnt

In [29]:
rushing_eng_stats['TD_per_Att'] = rushing_eng_stats['TD'] / rushing_eng_stats['Att']
rushing_eng_stats['Fmb_per_Att'] = rushing_eng_stats['Fmb'] / rushing_eng_stats['Att']
rushing_eng_stats['1D_per_Att'] = rushing_eng_stats['1D'] / rushing_eng_stats['Att']
rushing_eng_stats['TD_per_Game'] = rushing_eng_stats['TD'] / rushing_eng_stats['G']
rushing_eng_stats['Fmb_per_Game'] = rushing_eng_stats['Fmb'] / rushing_eng_stats['G']
rushing_eng_stats['TD_to_Fmb'] = rushing_eng_stats['TD'] / rushing_eng_stats['Fmb'].replace(0, np.nan)  # to avoid dividing by zero
rushing_eng_stats['Starter_Ratio'] = rushing_eng_stats['GS'] / rushing_eng_stats['G']
rushing_eng_stats['Lng_Yds_ratio'] = rushing_eng_stats['Lng'] / rushing_eng_stats['Yds'].replace(0, np.nan)

In [30]:
rushing_eng_stats.head()

,Player,Age,Team,G,GS,Att,Yds,TD,1D,Succ%,...,A/G,Fmb,TD_per_Att,Fmb_per_Att,1D_per_Att,TD_per_Game,Fmb_per_Game,TD_to_Fmb,Starter_Ratio,Lng_Yds_ratio
0,Saquon Barkley,27.0,PHI,16.0,16.0,345.0,2005.0,13.0,82.0,52.5,...,21.6,2.0,0.037681,0.005797,0.237681,0.812500,0.125000,6.500000,1.000000,0.035910
1,Derrick Henry,30.0,BAL,17.0,17.0,325.0,1921.0,16.0,94.0,58.8,...,19.1,3.0,0.049231,0.009231,0.289231,0.941176,0.176471,5.333333,1.000000,0.045289
2,Kyren Williams,24.0,LAR,16.0,16.0,316.0,1299.0,14.0,85.0,50.9,...,19.8,5.0,0.044304,0.015823,0.268987,0.875000,0.312500,2.800000,1.000000,0.023095
3,Bijan Robinson,22.0,ATL,17.0,17.0,304.0,1456.0,14.0,82.0,60.2,...,17.9,1.0,0.046053,0.003289,0.269737,0.823529,0.058824,14.000000,1.000000,0.025412
4,Jonathan Taylor,25.0,IND,14.0,13.0,303.0,1431.0,11.0,71.0,45.9,...,21.6,4.0,0.036304,0.013201,0.234323,0.785714,0.285714,2.750000,0.928571,0.048917


Now, to do our analysis with only our own engineered stats, lets drop all of the basic stats from before

In [31]:
for col in rushing_eng_stats.columns:
    if "_" not in col and col not in ["Player", "Team"]:
        rushing_eng_stats = rushing_eng_stats.drop(columns=[col])
rushing_eng_stats.columns

Index(['Player', 'Team', 'TD_per_Att', 'Fmb_per_Att', '1D_per_Att',
       'TD_per_Game', 'Fmb_per_Game', 'TD_to_Fmb', 'Starter_Ratio',
       'Lng_Yds_ratio'],
      dtype='object')

In [ ]:
NFL_RB_Identifiers = rushing_eng_stats[['Player', 'Team']]
NFL_RB_Features = rushing_eng_stats.drop(columns=['Player', 'Team'])

#NFL_RB_Identifiers.to_csv("C:/Users/jakel/OneDrive/Desktop/GitHub/NFL-Player-Clustering/filtered-data/NFL_RB_Identifiers.csv")
#NFL_RB_Features.to_csv("C:/Users/jakel/OneDrive/Desktop/GitHub/NFL-Player-Clustering/filtered-data/NFL_RB_Features.csv")